<a href="https://colab.research.google.com/github/emish8/Converting-Photos-to-Monet-Style-Paintings-Using-GANs/blob/main/calculate_fid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FID


In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from scipy.linalg import sqrtm
from PIL import Image

def get_activations(images, module, batch_size):
    """
    Get activations for a set of images using a given Inception V3 module.
    """
    activations = []
    for i in range(0, len(images), batch_size):
        batch = tf.convert_to_tensor(images[i:i+batch_size], dtype=tf.float32)
        activations.append(module(batch))
    activations = tf.concat(activations, axis=0)
    return activations


def calculate_fid(real_image_path, fake_image_folder_path, batch_size=1):
    module = hub.load('https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5')

    # Load up to 190 real images
    real_image_filenames = os.listdir(real_image_path)
    num_real_images = min(len(real_image_filenames), 190)
    real_image_filenames = real_image_filenames[:num_real_images]
    real_image_paths = [os.path.join(real_image_path, filename) for filename in real_image_filenames]
    real_images = np.array([np.array(Image.open(path)) for path in real_image_paths])

    # Load up to 190 fake images from the folder
    fake_image_filenames = os.listdir(fake_image_folder_path)
    num_fake_images = min(len(fake_image_filenames), 190)
    fake_image_filenames = fake_image_filenames[:num_fake_images]
    fake_image_paths = [os.path.join(fake_image_folder_path, filename) for filename in fake_image_filenames]
    fake_images = np.array([np.array(Image.open(path)) for path in fake_image_paths])

    # Compute mean and covariance of real images
    real_activations = get_activations(real_images, module, batch_size)
    real_mean = np.mean(real_activations, axis=0)
    real_cov = np.cov(real_activations, rowvar=False)

    # Compute mean and covariance of generated images
    fake_activations = get_activations(fake_images, module, batch_size)
    fake_mean = np.mean(fake_activations, axis=0)
    fake_cov = np.cov(fake_activations, rowvar=False)

    # Calculate FID
    mean_diff = real_mean - fake_mean
    cov_product = real_cov.dot(fake_cov)
    cov_sqrt = sqrtm(cov_product).real
    if np.iscomplexobj(cov_sqrt):
        cov_sqrt = cov_sqrt.real
    fid = mean_diff.dot(mean_diff) + np.trace(real_cov + fake_cov - 2 * cov_sqrt)

    return fid




In [2]:
real = '/content/drive/MyDrive/GAN_data/gan-getting-started/jpg/monet_jpg'
coverted_by_pix = '/content/drive/MyDrive/GAN_data/pix'
converted_by_cyclegan = '/content/drive/MyDrive/GAN_data/cycle'
converted_by_neural= '/content/drive/MyDrive/GAN_data/neural'

In [3]:
fid_pix2pix = calculate_fid(real, coverted_by_pix, batch_size=1)
print(fid_pix2pix)

1917342.134855315


In [4]:
fid_cycleGan = calculate_fid(real, converted_by_cyclegan , batch_size=1)
print(fid_cycleGan)

1199486.1822070514


In [20]:
import cv2
img = cv2.imread('/content/drive/MyDrive/GAN_data/neural/00.12757726104131295stylized-image.png')
img1= cv2.imread('/content/drive/MyDrive/GAN_data/cycle/0.jpg')

In [21]:
img.shape

(256, 256, 3)

In [22]:
fid_neural = calculate_fid(real, converted_by_neural, batch_size=1)
print(fid_neural)

208174.26033732155


In [23]:
import math

def calculate_mifid(real_image_path, fake_image_folder_path,   batch_size=1):
    # Calculate FID
    fid = calculate_fid(real_image_path, fake_image_folder_path, batch_size)

    # Compute scaling factor
    num_real_images = len(os.listdir(real_image_path))
    num_fake_images = len(os.listdir(fake_image_folder_path))
    scale = math.sqrt(num_real_images / num_fake_images)

    # Calculate MiFID
    mifid = fid * scale

    return mifid


In [24]:
mifid_pix2pix = calculate_mifid(real, coverted_by_pix, batch_size=20)
print(mifid_pix2pix)

2402943.497395592


In [ ]:
mifid_cycleGan = calculate_mifid(real,converted_by_cyclegan , batch_size=1)
print(mifid_cycleGan)

In [ ]:
mifid_neural = calculate_mifid(real,converted_by_neural, batch_size=1)
print(mifid_neural)

In [ ]:
from PIL import Image
import os

def resize_images(folder_path=converted_by_neural):
    # Get all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]
    i = 0
    # Loop through each image file
    for image_file in image_files:
        # Open the image file
        i +=1
        image_path = os.path.join(folder_path, image_file)
        with Image.open(image_path) as image:
            # Resize the image to (256,256) shape
            resized_image = image.resize((256, 256))

            # Convert the image to RGB mode if it's not already
            if resized_image.mode != 'RGB':
                resized_image = resized_image.convert('RGB')
            print(i, 'iteration')
            # Save the resized image
            resized_image.save(image_path)
    print('done')

In [19]:
resize_images()

1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
11 iteration
12 iteration
13 iteration
14 iteration
15 iteration
16 iteration
17 iteration
18 iteration
19 iteration
20 iteration
21 iteration
22 iteration
23 iteration
24 iteration
25 iteration
26 iteration
27 iteration
28 iteration
29 iteration
30 iteration
31 iteration
32 iteration
33 iteration
34 iteration
35 iteration
36 iteration
37 iteration
38 iteration
39 iteration
40 iteration
41 iteration
42 iteration
43 iteration
44 iteration
45 iteration
46 iteration
47 iteration
48 iteration
49 iteration
50 iteration
51 iteration
52 iteration
53 iteration
54 iteration
55 iteration
56 iteration
57 iteration
58 iteration
59 iteration
60 iteration
61 iteration
62 iteration
63 iteration
64 iteration
65 iteration
66 iteration
67 iteration
68 iteration
69 iteration
70 iteration
71 iteration
72 iteration
73 iteration
74 iteration
75 iteration
76 iteration
77 iteration
78 itera